# ハイブリッド検索について体験してみよう

Knowledge bases for Amazon Bedrock では、ベクトルデータベースが OpenSearch Serverless の場合、 ベクトル検索に加えキーワード検索を行う、ハイブリッド検索をサポートしています。

OpenSearch Serverless の場合、次のクエリオプションが選択できます。

- Default
  - Bedrock 側で、ベクターデータベースの構成に最適な検索方法（Hybrid or Semantic）を決定してクエリを実行
- Hybrid
  - ベクトル検索とキーワード検索の両方を利用してクエリを実行
- Semantic
  - ベクトル検索のみでクエリを実行

この章ではハイブリッド検索を利用して、回答制度の向上を試してみます。

サンプルドキュメントの `all` フォルダには社内の申請に必要なドキュメントを格納しています。それぞれの申請には独自のツールを利用する必要があることが明記されています。

- 人事君：住所変更を伴うための社内ツール
- 労務君：給与口座変更を行うための社内ツール
- 経理君：経費清算を行うための社内ツール

## RetrieveAndGenerate API

In [ ]:
import json
import boto3

KNOWLEDGEBASE_ID = "" # ナレッジベース ID を記載
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_agent = boto3.client("bedrock-agent-runtime")
prompt = "労務君は何をするものなのか教えてください。"

response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": 'KNOWLEDGE_BASE',
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn
        },
    },
)

print(response["output"]["text"])